In [18]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import dask.dataframe as dd
import sqlite3
from scipy import sparse


import requests
import json
import aiohttp
import asyncio
import nest_asyncio

nest_asyncio.apply()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [44]:
paper_table_file = "../data/testData.csv"

ES_PASSWORD = "FSailing4046"
ES_USERNAME = "skojaku"
ES_ENDPOINT = "localhost:9200/" # wos/_search/

In [8]:
paper_table = pd.read_csv(paper_table_file)
paper_table.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,xref_doi,art_no,UID,WoSauthors,WoSjournal,score_title_0,score_author_0,ID_researcher,Author_name,author_order
0,3jd27e11,NaN,WHO,Nursing workload in the COVID-19 ERA,NaN,NaN,NaN,unk,NaN,2020,...,10.1111/j.0737-1209.2005.220504.x,NaN,WOS:000232556900004,"['Cava, MA', 'Fay, KE', 'Beanlands, HJ', 'McCa...",PUBLIC HEALTH NURSING,33.0,-1.0,28313,McCay-EA,4
1,449ggozj,NaN,WHO,Failure in initial stage containment of global...,10.1038/nature03712,NaN,NaN,unk,"With multiple virus epicenters, COVID-19 has b...",2020,...,NaN,NaN,WOS:000230296600049,"['Imai, Y', 'Kuba, K', 'Rao, S', 'Huan, Y', 'G...",NATURE,36.0,-1.0,13366;23486;23609;25090;28576;29376,Slutsky-AS;Hui-CC;Penninger-JM;LeongPoi-H;Crac...,16;13;18;10;11;1
2,6b35vzd8,NaN,WHO,Carpal Fracture and COVID-19 Infection: Observ...,10.1016/j.ajic.2004.11.003,NaN,NaN,unk,NaN,2020,...,NaN,NaN,WOS:000227535000004,"['Moore, D', 'Gamage, B', 'Bryce, E', 'Copes, ...",AMERICAN JOURNAL OF INFECTION CONTROL,34.0,-1.0,23322;32091,Yassi-A;Moore-D,5;1
3,7b4ssyzm,NaN,Elsevier; PMC,Prevention of EAE in non-human primates by a t...,NaN,PMC7119632,NaN,els-covid,NaN,1994-10-31,...,NaN,NaN,WOS:A1994NY34200043,"['JOLICOEUR, P', 'LAMONTAGNE, L']",JOURNAL OF IMMUNOLOGY,38.0,-1.0,3884,LAMONTAGNE-L,2
4,7v1ng0ne,NaN,WHO,Editorial: The Year in Review Amidst the COVID...,10.1146/annurev.med.56.091103.134135,NaN,NaN,unk,NaN,2020,...,NaN,NaN,WOS:000227504100020,"['Skowronski, DA', 'Astell, C', 'Brunham, RC',...",ANNUAL REVIEW OF MEDICINE,40.0,-1.0,6335;13497;13506;23318;23882;32261,Talbot-PJ;Babiuk-L;Brunham-RC;Skowronski-DA;Lo...,7;9;3;1;4;5


In [46]:
# Set the end point for the ElasticSearch
es_end_point = "https://{endpoint}".format(
    user=ES_USERNAME, password=ES_PASSWORD, endpoint=ES_ENDPOINT
)
print(es_end_point)
res = requests.get(es_end_point)
print(res.content)
# es = Elasticsearch()

https://localhost:9200/


ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

# Create tables

In [23]:
paper_table["UID"]

0       WOS:000232556900004
1       WOS:000230296600049
2       WOS:000227535000004
3       WOS:A1994NY34200043
4       WOS:000227504100020
               ...         
5858    WOS:000231514200008
5859    WOS:000231514200008
5860    WOS:000231514200008
5861    WOS:000235218000001
5862    WOS:000228553800006
Name: UID, Length: 5863, dtype: object

In [29]:
wos_ids = paper_table["UID"].values

query = '{"ids": "values": [%s]}' % ",".join(wos_ids[:4])
res = es.search(index="test-index", body={"query": {"match_all": {}}})

NameError: name 'es' is not defined